Thinking1: 关联规则中的支持度、置信度和提升度代表的什么，如何计算?
   - 支持度：某件物品或者某组物品出现频率： $supp(A) = 包含A的交易数/总交易数$
   - 置信度：在有A的交易中，出现B的频率：$conf(A\Rightarrow B) = \frac{supp(A\cup B)}{supp(A)}$ 
   - 提升度：在有A的交易中，B的频率与B本身的频率的比值：$lift(A\Rightarrow B) = \frac{conf(A\Rightarrow B)}{supp(B)} = \frac{supp(A\cup B)}{supp(A)supp(B}$
   
   
Thinking2: 关联规则与协同过滤的区别
   - 关联规则利用所有交易的信息，关注全局，是静态的
   - 协同过滤则根据用户行为，关注的是某个用户或物品，是动态的
   
   
Thinking3：为什么我们需要多种推荐算法？
   - 每种推荐算法有其适用场景，没有一种推荐算法能够满足所有的应用场景
   - 不同推荐算法的优劣势，所关注的指标不同，而且不同用户可能需要适配不同指标，一个优秀的推荐系统需要同时关注多种指标
   - 在不同时期，推荐系统对推荐算法的需求也不一样，从冷启动到基于用户历史行为等。

Thinking4：关联规则中的最小支持度、最小置信度该如何确定？
   - 通过实验不断调整。
   - 先设定比较低，得到较多的频繁项集，然后对每个频繁项集的支持度进行排序，选择满意的频繁项集个数对应的支持度阈值。
   - 置信度同理，先得到较多的关联规则，然后对每个关联规则的置信度排序，选择满意的关联规则个数对应的置信度阈值。

Thinking5：都有哪些常见的回归分析方法，评价指标是什么？
   - 一元线性回归、多元线性回归、多项式回归
   - 评价指标：R-square（确定性系数）

Action1: 针对MarketBasket数据集进行购物篮分析（频繁项集及关联规则挖掘）

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


# 读取数据
dataset = pd.read_csv('./data/Market_Basket_Optimisation.csv', header = None)
# 构建交易数据
transactions = []
# 得到所有物品集合，为后面传入apriori的数据做准备
items = set()
for i in range(dataset.shape[0]):
    trans = set()
    for j in range(dataset.shape[1]):
        trans.add(str(dataset.values[i, j]))
        items.add(str(dataset.values[i, j]))
    trans.discard('nan')
    transactions.append(trans)
# 数据转换：转换成mlxtend的数据输入
cols = list(items)
col_idx = {col:idx for idx, col in enumerate(cols)}
# print(col_idx)
# 初始化输入的data
data = np.zeros(shape=(len(transactions), len(cols)))
for i in range(len(transactions)):
    for item in transactions[i]:
        data[i][col_idx[item]] = 1
# 得到符合输入格式的交易数据
transactions_df = pd.DataFrame(data, columns=cols)

# 频繁项集
frequent_itemsets = apriori(transactions_df, min_support=0.05, use_colnames=True)
print('frequent itemsets:\n', frequent_itemsets)
# 关联规则
# 提升度作为度量参数
rules_lift = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print('association rules using lift:\n', rules_lift)
# 置信度作为度量参数，是该函数metric的默认参数
rules_conf = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
print('association rules using confidence:\n', rules_conf)